###HW4.0.3.
Define big data. Provide an example of a big data problem in your domain of expertise.
What is a race condition in the context of parallel computation? Give an example. What is MapReduce? How does it differ from Hadoop?

Data that is large in size, continuously changing and is potentially unstructured is referred to as Big Data. Traditional data processing method are inadequate while working with big data and hence special methods are required.

Energy consumption data recorded through the sensors is an example of big data. The data getting streamed continuously at one minute interval can provide a lot information around equipment usage and the peak consumption.

A race condition is a parallel programming fault producing undetermined program state and behavior due to un-synchronized parallel program executions. It can occur when two threads receive the same data and try to change it simultaneously

CASE 1: x = x+1 –A and B could both read the initial value of X and then overwrite each other –FINAL RESULT: x = x+1 CASE 2: x = x+2 –A reads and writes, then B reads and writes –FINAL RESULT: x = x + 2 •That’s a race condition

Map Reduce is a programming model that deals with large complex data through
a)Distributed data handling and storage 
b)Distributed computation : Move the code to the data 

Hadoop is an open source implementation of Map- Reduce that handles the processing details

## 3.  HW4.2 WordCount of SPAM data  <a name="1.2"></a>
[Back to Table of Contents](#TOC)

HW4.2:  WORDCOUNT
Using the Enron dataset and MRJob, write the mapper/reducer job that  will determine the word count (number of occurrences) of each white-space delimitted token (assume spaces, fullstops, comma as delimiters). Examine the word “assistance” and report its word count results.

 

In [149]:
%%writefile MRWordCount.py

from mrjob.job import MRJob
import re
import sys
import operator

word_re = re.compile(r"[\w']+")

class MRWordCounter(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(MRWordCounter, self).__init__(*args, **kwargs)
        self.wordcount = {}
        
    def mapper(self,_,line):
        docID, docClass,text = line.split("\t",2)
        text = text.strip()
        for word in word_re.findall(text):
            if word[0]>='a' and word[0]<='z':
                yield word.lower(),1

    def reducer(self,word,counts):
        if word == 'assistance':
            print "The word",word, "occurs",sum(counts),"times"
        self.wordcount[word] = sum(counts)
        yield(word, self.wordcount[word])

if __name__=='__main__':
    MRWordCounter.run()


Overwriting MRWordCount.py


In [150]:
!python MRWordCount.py "enronemail_1h.txt"

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/MRWordCount.z071103.20160905.180813.585222
Running step 1 of 1...
The word assistance occurs 10 times
Streaming final output from /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/MRWordCount.z071103.20160905.180813.585222/output...
"a"	542
"a'click"	1
"ab"	5
"abidjan"	2
"ability"	2
"able"	14
"abn"	1
"about"	52
"above"	11
"absent"	1
"absenteeism"	1
"absolute"	2
"absolutely"	1
"absorb"	1
"abuse"	2
"abused"	1
"acce"	1
"accelerate"	1
"accelerated"	1
"accept"	3
"acceptable"	1
"accepted"	1
"accepting"	2
"accepts"	1
"access"	12
"accomodate"	4
"accomodates"	1
"accompanied"	1
"according"	2
"accordingly"	1
"account"	36
"accountability"	1
"accounting"	5
"accounts"	1
"accrual"	2
"accurate"	1
"aches"	1
"achieve"	1
"achieved"	1
"acid"	1
"acquire"	1
"acquisition"	1
"acrobaat"	1
"acrobat"	1
"across"	10
"act"	7
"action"	1
"activate"	4
"active"	1
"activists"	1
"activities"	10
"ac

In [3]:
!grep assistance enronemail_1h.txt|cut -d$'\t' -f4| grep assistance|wc -l 

       8


### HW4.2.1  Total Sort using a single reducer
Using MrJob determine the top-10 occurring tokens (most frequent tokens) using a single reducer. 

In [85]:
%%writefile MRWordCountSorted.py

from mrjob.job import MRJob
import re
import sys
import operator

word_re = re.compile(r"[\w']+")

class MRWordCountSorted(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(MRWordCountSorted, self).__init__(*args, **kwargs)
        self.wordcount = {}
    
    def mapper(self,_,line):
        docID, docClass,text = line.split("\t",2)
        text = text.strip()
        for word in word_re.findall(text):
            if word[0]>='a' and word[0]<='z':
                yield word.lower(),1
                
    def reducer(self,word,counts):
        self.wordcount[word] = sum(counts)
        yield(word, self.wordcount[word])

    def reducer_final(self):
        top_ten = sorted(self.wordcount.items(), key=operator.itemgetter(1),reverse=True)[:10]
        print "The top 10 words are-",(top_ten)

    def jobconf(self):
        orig_jobconf = super(MRWordCountSorted, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k2,2rn -k1,1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf

if __name__=='__main__':
    MRWordCountSorted.run()


Overwriting MRWordCountSorted.py


In [86]:
!chmod a+x MRWordCountSorted.py

In [88]:
!python MRWordCountSorted.py "enronemail_1h.txt"

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/MRWordCountSorted.z071103.20160905.102306.449204
Running step 1 of 1...
The top 10 words are- [(u'the', 1247), (u'to', 963), (u'and', 668), (u'of', 566), (u'a', 542), (u'you', 432), (u'in', 417), (u'your', 394), (u'ect', 382), (u'for', 373)]
Streaming final output from /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/MRWordCountSorted.z071103.20160905.102306.449204/output...
"a"	542
"a'click"	1
"ab"	5
"abidjan"	2
"ability"	2
"able"	14
"abn"	1
"about"	52
"above"	11
"absent"	1
"absenteeism"	1
"absolute"	2
"absolutely"	1
"absorb"	1
"abuse"	2
"abused"	1
"acce"	1
"accelerate"	1
"accelerated"	1
"accept"	3
"acceptable"	1
"accepted"	1
"accepting"	2
"accepts"	1
"access"	12
"accomodate"	4
"accomodates"	1
"accompanied"	1
"according"	2
"accordingly"	1
"account"	36
"accountability"	1
"accounting"	5
"accounts"	1
"accrual"	2
"accurate"	1
"aches"	1
"achieve"	1
"achieved"	1
"aci

### HW4.2.2 (Optional)  Total Sort using MULTIPLE reducers
Using MrJob determine the top-10 occurring tokens (most frequent tokens) using three (3) reducers. HINT: * Check out the following notebook for examples of total sort using multiple reducers in MRJob [Total Sort Notebook for Hadoop Streaming and for MRJob](http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/44idk0xz2wyu45l/total-sort-guide.ipynb) 


## 3.  HW4.3  Multinomial NAIVE BAYES with NO Smoothing using a single reducer
<a name="1.3"></a>
[Back to Table of Contents](#TOC)

Using the Enron data from HW1 and Hadoop MapReduce (or MRJob), write  a mapper/reducer job(s) that
   will both learn  Naive Bayes classifier and classify the Enron email messages using the learnt Naive Bayes classifier. Use all white-space delimitted tokens as independent input variables (assume spaces, fullstops, commas as delimiters). Note: for multinomial Naive Bayes, the Pr(X=“assistance”|Y=SPAM) is calculated as follows:

   the number of times “assistance” occurs in SPAM labeled documents / the number of words in documents labeled SPAM 

   E.g.,   “assistance” occurs 5 times in all of the documents Labeled SPAM, and the length in terms of the number of words in all documents labeled as SPAM (when concatenated) is 1,000. Then Pr(X=“assistance”|Y=SPAM) = 5/1000. Note this is a multinomial estimation of the class conditional for a Naive Bayes Classifier. No smoothing is needed in this HW. Multiplying lots of probabilities, which are between 0 and 1, can result in floating-point underflow. Since log(xy) = log(x) + log(y), it is better to perform all computations by summing logs of probabilities rather than multiplying probabilities. Please pay attention to probabilites that are zero! They will need special attention. Count up how many times you need to process a zero probabilty for each class and report. 

   Report the performance of your learnt classifier in terms of misclassifcation error rate of your multinomial Naive Bayes Classifier. Plot a histogram of the  posterior probabilities (i.e., Pr(Class|Doc)) for each class over the training set. Summarize what you see. 

   Error Rate = misclassification rate with respect to a provided set (say training set in this case. This is not a good practice in real time!). It is more formally defined here:

Let DF represent the evalution set in the following:
Err(Model, DF) = |{(X, c(X)) ∈ DF : c(X) != Model(x)}|   / |DF|

Where || denotes set cardinality; c(X) denotes the class of the tuple X in DF; and Model(X) denotes the class inferred by the Model “Model”

NOTE: please assume one reducer.

In [41]:
%%writefile NBnosmooth_Trainer.py

from mrjob.job import MRJob
import sys, re

def clean_text(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    text= re.sub('[0-9]+',' ',text)
    words = text.split()
    for word in words:
         cleaned.append(word)
    return cleaned

class NBnosmooth_Trainer(MRJob):
          
    def __init__(self, *args, **kwargs):
        super(NBnosmooth_Trainer, self).__init__(*args, **kwargs)
        self.nb_wordstats = {}
        
    def mapper(self,_,line):
        docID, docClass,text = line.split("\t",2)
        words = clean_text(text)
        if docClass == "1":
            yield("PriorsNB","1,0")
            for word in words:
                yield (word, "1,0")        
        else:
            yield("PriorsNB","0,1")
            for word in words:
                yield (word, "0,1")
    
    def reducer(self,word,values):
        spm_total = 0
        not_spm_total = 0
        for value in values:
            spm_count, not_spm_count = value.split(",")
            spm_total +=float(spm_count)
            not_spm_total+=float(not_spm_count)
        self.nb_wordstats[word] = [spm_total,not_spm_total]
    
    def reducer_final(self):
        #1.get total wc in spam and not in spam
        spam_words_total = 0
        notspam_words_total = 0
        spam_docs_total = 0
        notspam_docs_total = 0
        total_docs = 0
        for word in self.nb_wordstats.keys():
            if word !="PriorsNB":
                spam_words_total+=self.nb_wordstats[word][0]
                notspam_words_total+=self.nb_wordstats[word][1]
            else:
        #2. Calculate and yield prior info from total number of spam and not spam documents and total documents
                spam_docs_total, notspam_docs_total = self.nb_wordstats.get(word)
                total_docs = spam_docs_total + notspam_docs_total
                yield("PriorsNB", ','.join(str(j) for j in [spam_docs_total/total_docs, notspam_docs_total/total_docs])) 
                del self.nb_wordstats[word]
        #3.Get proportion of each word with respect to total word count in spam or not spam
        for word in self.nb_wordstats.keys():
            yield(word, ','.join(str(j) for j in [self.nb_wordstats[word][0]/spam_words_total, self.nb_wordstats[word][1]/notspam_words_total])) 
    
    def jobconf(self):
        orig_jobconf = super(NBnosmooth_Trainer, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k2,2rn -k1,1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf  
    
if __name__=='__main__':
    NBnosmooth_Trainer.run()           

Overwriting NBnosmooth_Trainer.py


In [34]:
!python NBnosmooth_Trainer.py "enronemail_1h.txt" 

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NBnosmooth_Trainer.z071103.20160905.060323.715714
Running step 1 of 1...
Streaming final output from /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NBnosmooth_Trainer.z071103.20160905.060323.715714/output...
"PriorsNB"	"0.44,0.56"
"stock"	"0.000110357004911,0.00022453409176"
"limited"	"0.000551785024554,0.000299378789013"
"entergyr"	"0.0,7.48446972532e-05"
"additionally"	"5.51785024554e-05,7.48446972532e-05"
"doubts"	"5.51785024554e-05,0.0"
"dynamic"	"0.0,0.000149689394506"
"externally"	"0.000110357004911,0.0"
"yellow"	"5.51785024554e-05,0.0"
"barraged"	"5.51785024554e-05,0.0"
"prefix"	"5.51785024554e-05,0.0"
"sleep"	"0.000110357004911,0.0"
"invovled"	"0.0,0.000149689394506"
"railing"	"5.51785024554e-05,0.0"
"appetite"	"0.000110357004911,0.0"
"hate"	"0.000220714009822,0.0"
"worked"	"0.000110357004911,7.48446972532e-05"
"forget"	"5.51785024554e-05,0.0"
"looking

In [153]:
%reload_ext autoreload
%autoreload 2

from numpy import random
from NBnosmooth_Trainer import NBnosmooth_Trainer 

# STEP 1: Train a mulitnomial Naive Bayes      
trainingData = 'enronemail_1h.txt'

# create an instance of the Trainer class
# and initiatialize it
mr_job = NBnosmooth_Trainer(args=[trainingData])
modelStats={}
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access to the output reducer/reducer_final of 
    # the last step in MRNaiveBayesTrainer
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key, value
        modelStats[key] = value            
    with open('NB_Model.txt', 'w') as f:
        for k in modelStats.keys():
            f.writelines( k + "\t"+ str(modelStats[k]) +"\n")
print modelStats

PriorsNB 0.44,0.56
stock 0.000110357004911,0.00022453409176
limited 0.000551785024554,0.000299378789013
entergyr 0.0,7.48446972532e-05
additionally 5.51785024554e-05,7.48446972532e-05
doubts 5.51785024554e-05,0.0
dynamic 0.0,0.000149689394506
externally 0.000110357004911,0.0
yellow 5.51785024554e-05,0.0
barraged 5.51785024554e-05,0.0
prefix 5.51785024554e-05,0.0
sleep 0.000110357004911,0.0
invovled 0.0,0.000149689394506
railing 5.51785024554e-05,0.0
appetite 0.000110357004911,0.0
hate 0.000220714009822,0.0
worked 0.000110357004911,7.48446972532e-05
forget 5.51785024554e-05,0.0
looking 0.000220714009822,0.0
fronts 0.0,7.48446972532e-05
founder 5.51785024554e-05,0.0
granting 5.51785024554e-05,0.0
electricity 0.0,7.48446972532e-05
turner 0.000110357004911,7.48446972532e-05
locomotive 5.51785024554e-05,0.0
originality 5.51785024554e-05,0.0
ilug 0.000165535507366,0.0
koromah 0.000110357004911,0.0
swap 0.000110357004911,0.0
whatsoever 0.000165535507366,0.0
riedel 0.0,7.48446972532e-05
sibila

In [154]:
%%writefile NB_Classifier.py
 
from mrjob.job import MRJob
import sys, re, string, operator, math, os

def clean_text(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    text= re.sub('[0-9]+',' ',text)
    words = text.split()
    for word in words:
         cleaned.append(word)
    return cleaned

class NB_Classifier(MRJob):

    def __init__(self, *args, **kwargs):
        super(NB_Classifier, self).__init__(*args, **kwargs)
        self.nb_wordstats={}
        self.classification_output ={}
        
    #load model, that is, word proportional table from file output of trainer
    def mapper_init(self):
        self.nb_wordstats = {}
        recordStrs = [s.split('\n')[0].split('\t') for s in open("NB_Model.txt").readlines()]
        for word, statsStr in recordStrs:
            self.nb_wordstats[word] = map(float, statsStr.split(","))
        #Get prior probabilities
        self.prior_Spam = math.log(self.nb_wordstats["PriorsNB"][0])
        self.prior_notSpam = math.log(self.nb_wordstats["PriorsNB"][1])
        
    def mapper(self,_,line):
        docID,docClass,text=line.split('\t',2)
        words = clean_text(text)
        for word in words:
            P_spam_word = math.log(self.nb_wordstats[word][0]) if self.nb_wordstats[word][0]>0.0 else math.log(1)
            P_notspam_word = math.log(self.nb_wordstats[word][1]) if self.nb_wordstats[word][1]>0.0 else math.log(1)
        P_spam_givenwords = P_spam_word + self.prior_Spam        
        P_notspam_givenwords = P_notspam_word + self.prior_notSpam
        if P_spam_givenwords > P_notspam_givenwords:
            class_predicted = 1 
        else :
            class_predicted = 0
        yield('Total_Docs', '0,1')
        if int(docClass) != class_predicted: yield ('Misclassification', '0,1')  
        yield(docID, ','.join(str(j) for j in [docClass, class_predicted]))
    
    def reducer(self,docID,values):
        Total_counter =  0
        for value in values:
            correct,info = value.split(',')
            Total_counter +=int(info)
        self.classification_output[docID] = Total_counter

    def reducer_final(self):
        print 'Total number of docs is  ', self.classification_output.get("Total_Docs")
        print 'Error rate is %f', float(self.classification_output.get("Misclassification"))/float(self.classification_output.get("Total_Docs"))

    def jobconf(self):
        orig_jobconf = super(NB_Classifier, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf      
        
if __name__ == '__main__':
    NB_Classifier.run()    

Overwriting NB_Classifier.py


In [155]:
!python NB_Classifier.py --jobconf mapred.reduce.tasks=1 enronemail_1h.txt --file=NB_Model.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.183850.875536
Running step 1 of 1...
Total number of docs is   100
Error rate is %f 0.73
Streaming final output from /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.183850.875536/output...
Removing temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.183850.875536...


## 3.  HW4.4 Multinomial Naive Bayes with Smoothing
<a name="1.4"></a>
[Back to Table of Contents](#TOC)

Repeat HW4.3 with the following modification: use Laplace plus-one smoothing. Compare the misclassifcation error rates for HW4.3 versus HW1.4 and explain the differences.

For a quick reference on the construction of the Multinomial NAIVE BAYES classifier that you will code,
please consult the "Document Classification" section of the following wikipedia page:

https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Document_classification

OR the original paper by the curators of the Enron email data:

http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf

In [47]:
%%writefile NBLaplace_Trainer.py

from mrjob.job import MRJob
import sys, re

def clean_text(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    text= re.sub('[0-9]+',' ',text)
    words = text.split()
    for word in words:
         cleaned.append(word)
    return cleaned

class NBLaplace_Trainer(MRJob):
          
    def __init__(self, *args, **kwargs):
        super(NBLaplace_Trainer, self).__init__(*args, **kwargs)
        self.nb_wordstats = {}

   
    def mapper(self,_,line):
        docID, docClass,text = line.split("\t",2)
        words = clean_text(text)
        if docClass == "1":
            yield("PriorsNB","1,0")
            for word in words:
                yield (word, "1,0")        
        else:
            yield("PriorsNB","0,1")
            for word in words:
                yield (word, "0,1")
    
    def reducer(self,word,values):
        spm_total =0
        not_spm_total = 0
        for value in values:
            spm_count, not_spm_count = value.split(",")
            spm_total +=float(spm_count)
            not_spm_total+=float(not_spm_count)
        self.nb_wordstats[word] = [spm_total,not_spm_total]
    
    def reducer_final(self):
        
        spam_words_total = notspam_words_total = spam_docs_total = notspam_docs_total = total_docs = 0
        for word in self.nb_wordstats.keys():
            if word !="PriorsNB":
                spam_words_total+=self.nb_wordstats[word][0]
                notspam_words_total+=self.nb_wordstats[word][1]
            else:
        
                spam_docs_total, notspam_docs_total = self.nb_wordstats.get(word)
                total_docs = spam_docs_total + notspam_docs_total
                yield("PriorsNB", ','.join(str(j) for j in [spam_docs_total/total_docs, notspam_docs_total/total_docs])) 
                del self.nb_wordstats[word]
        
        unique_words_total = len(self.nb_wordstats.keys())
        
        for word in self.nb_wordstats.keys():
            yield(word, ','.join(str(j) for j in [(self.nb_wordstats[word][0]+1)/(spam_words_total+unique_words_total), (self.nb_wordstats[word][1]+1)/(notspam_words_total+unique_words_total)])) 
         
    
     def jobconf(self):
        orig_jobconf = super(NBLaplace_Trainer, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k2,2rn -k1,1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
        
if __name__=='__main__':
    NBLaplace_Trainer.run()           

Overwriting NBLaplace_Trainer.py


In [48]:
%reload_ext autoreload
%autoreload 2

from numpy import random
from NBLaplace_Trainer import NBLaplace_Trainer 
     
trainingData = 'enronemail_1h.txt'

# create an instance of the Trainer class
# and initiatialize it
mr_job = NBLaplace_Trainer(args=[trainingData])
modelStats={}
with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key, value
        modelStats[key] = value            
    with open('NB_Model.txt', 'w') as f:
        for k in modelStats.keys():
            f.writelines( k + "\t"+ str(modelStats[k]) +"\n")
print modelStats

PriorsNB 0.44,0.56
stock 0.000129332643559,0.00021699034393
limited 0.000474219693051,0.000271237929912
entergyr 4.31108811864e-05,0.000108495171965
additionally 8.62217623728e-05,0.000108495171965
doubts 8.62217623728e-05,5.42475859824e-05
dynamic 4.31108811864e-05,0.000162742757947
externally 0.000129332643559,5.42475859824e-05
yellow 8.62217623728e-05,5.42475859824e-05
barraged 8.62217623728e-05,5.42475859824e-05
prefix 8.62217623728e-05,5.42475859824e-05
sleep 0.000129332643559,5.42475859824e-05
invovled 4.31108811864e-05,0.000162742757947
railing 8.62217623728e-05,5.42475859824e-05
appetite 0.000129332643559,5.42475859824e-05
hate 0.000215554405932,5.42475859824e-05
worked 0.000129332643559,0.000108495171965
forget 8.62217623728e-05,5.42475859824e-05
looking 0.000215554405932,5.42475859824e-05
fronts 4.31108811864e-05,0.000108495171965
founder 8.62217623728e-05,5.42475859824e-05
granting 8.62217623728e-05,5.42475859824e-05
electricity 4.31108811864e-05,0.000108495171965
turner 0.0

In [49]:
!python NB_Classifier.py --jobconf mapred.reduce.tasks=1 enronemail_1h.txt --file=NB_Model.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.063336.921424
Running step 1 of 1...
Total number of docs is   100
Error rate is %f 0.18
Streaming final output from /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.063336.921424/output...
Removing temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.063336.921424...


### HW 4.4.1 Jelinek-Mercer (JM) smoothing* 

With different smoothing methods, p(wk|ci) (i.e., the word class conditionals) will be computed
differently. We consider Jelinek-Mercer (JM) smoothing as an alternative to Laplace  Let c(w, ci) denote
the frequency of word w in category ci,  p(w|C) be the maximum likelihood estimation of word w in 
collection C (relative frequency) and let |C for classi| denote the length of the classi. Then:

1) Jelinek-Mercer (JM) smoothing:
λp(w|ci) = (1 − λ) * c(w, ci)/sum_over_wJ_in_V(c(wJ, ci))    +  λ p(w|C)

Where c(w, ci)/sum_over_wJ_in_V(c(wJ, ci)) essential denotes the relative frequency of word w in class ci, i.e., Pr(w|ci)

where λ = 0.3  
λ a hyperparameter for the Jelinek-Mercer (JM) smoothing that gives more (or less) 
credit to the background model p(w|C) or to the foreground model 

Repleat HW 4.4.0 with Jelinek-Mercer (JM) smoothing. Present a nice table comparing results where each 
row is the approach taken (Multinomial Naive Bayes with Laplace+1, or Multinomial Naive Bayes with JM) and the column is error rate.

* REFERENCES 
   + http://www.ntu.edu.sg/home/gaocong/papers/wpp095-yuan.pdf

In [92]:
%%writefile NBJM_Trainer.py

from mrjob.job import MRJob
import sys, re

def clean_text(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    text= re.sub('[0-9]+',' ',text)
    words = text.split()
    for word in words:
         cleaned.append(word)
    return cleaned

class NBJM_Trainer(MRJob):
          
    def __init__(self, *args, **kwargs):
        super(NBJM_Trainer, self).__init__(*args, **kwargs)
        self.nb_wordstats = {}

  
    def mapper(self,_,line):
        docID, docClass,text = line.split("\t",2)
        words = clean_text(text)
        if docClass == "1":
            yield("PriorsNB","1,0")
            for word in words:
                yield (word, "1,0")        
        else:
            yield("PriorsNB","0,1")
            for word in words:
                yield (word, "0,1")
    
    def reducer(self,word,values):
        spm_total = 0
        not_spm_total = 0
        for value in values:
            spm_count, not_spm_count = value.split(",")
            spm_total +=float(spm_count)
            not_spm_total+=float(not_spm_count)
        self.nb_wordstats[word] = [spm_total,not_spm_total]   

    def reducer_final(self):
        #1.get total wc in spam and not in spam
        spam_words_total = 0
        notspam_words_total = 0
        spam_docs_total = 0
        notspam_docs_total = 0
        total_docs = 0
        JM = 0.3
        for word in self.nb_wordstats.keys():
            if word !="PriorsNB":
                spam_words_total+=self.nb_wordstats[word][0]
                notspam_words_total+=self.nb_wordstats[word][1]
            else:
        #2. Calculate and yield prior info from total number of spam and not spam documents and total documents
                spam_docs_total, notspam_docs_total = self.nb_wordstats.get(word)
                total_docs = spam_docs_total + notspam_docs_total
                yield("PriorsNB", ','.join(str(j) for j in [spam_docs_total/total_docs, notspam_docs_total/total_docs])) 
                del self.nb_wordstats[word]
        #4.Get JM proportion of each word 
        for word in self.nb_wordstats.keys():
            yield(word, ','.join(str(j) for j in [(((1-JM)*(self.nb_wordstats[word][0]/spam_words_total)) + (JM *(self.nb_wordstats[word][0] + self.nb_wordstats[word][1])/(spam_words_total + notspam_words_total))),(((1-JM)*(self.nb_wordstats[word][1]/notspam_words_total)) + (JM *(self.nb_wordstats[word][0] + self.nb_wordstats[word][1])/(spam_words_total + notspam_words_total)))])) 
         
    def jobconf(self):
        orig_jobconf = super(NBJM_Trainer, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k2,2rn -k1,1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
        
if __name__=='__main__':
    NBJM_Trainer.run()           

Overwriting NBJM_Trainer.py


In [93]:
%reload_ext autoreload
%autoreload 2

from numpy import random
from NBJM_Trainer import NBJM_Trainer 

# STEP 1: Train a mulitnomial Naive Bayes      
trainingData = 'enronemail_1h.txt'


mr_job = NBJM_Trainer(args=[trainingData])
modelStats={}
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access to the output reducer/reducer_final of 
    # the last step in MRNaiveBayesTrainer
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key, value
        modelStats[key] = value            
    with open('NB_Model.txt', 'w') as f:
        for k in modelStats.keys():
            f.writelines( k + "\t"+ str(modelStats[k]) +"\n")
print modelStats

PriorsNB 0.44,0.56
stock 0.000124893150801,0.000204817111595
limited 0.000519650609807,0.000342966244927
entergyr 9.52864947275e-06,6.191993755e-05
additionally 5.76822506643e-05,7.14485870227e-05
doubts 4.81536011916e-05,9.52864947275e-06
dynamic 1.90572989455e-05,0.0001238398751
externally 9.63072023831e-05,1.90572989455e-05
yellow 4.81536011916e-05,9.52864947275e-06
barraged 4.81536011916e-05,9.52864947275e-06
prefix 4.81536011916e-05,9.52864947275e-06
sleep 9.63072023831e-05,1.90572989455e-05
invovled 1.90572989455e-05,0.0001238398751
railing 4.81536011916e-05,9.52864947275e-06
appetite 9.63072023831e-05,1.90572989455e-05
hate 0.000192614404766,3.8114597891e-05
worked 0.000105835851856,8.09772364955e-05
forget 4.81536011916e-05,9.52864947275e-06
looking 0.000192614404766,3.8114597891e-05
fronts 9.52864947275e-06,6.191993755e-05
founder 4.81536011916e-05,9.52864947275e-06
granting 4.81536011916e-05,9.52864947275e-06
electricity 9.52864947275e-06,6.191993755e-05
turner 0.000105835851

In [94]:
!python NB_Classifier.py --jobconf mapred.reduce.tasks=1 enronemail_1h.txt --file=NB_Model.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.105025.857674
Running step 1 of 1...
Total number of docs is   100
Error rate is %f 0.17
Streaming final output from /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.105025.857674/output...
Removing temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.105025.857674...


###3. HW4.5 Remove words with frequency of less than three (3) in the training set

 This time when modeling and classification ignore tokens with a frequency of less than 
 three (3) in the training set. How does it affect the misclassifcation error of learnt naive multinomial
 Bayesian Classifier on the training dataset. Report the error and the change in error.

HINT: ignore tokens with a frequency of less than three (3). Think of this as a preprocessing step. How many new mapreduce jobs do you need to solve thus homework?

In [140]:
%%writefile WordstoKeep.py

from mrjob.job import MRJob
import re
import sys
import operator

word_re = re.compile(r"[\w']+")

class Stopwords(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(Stopwords, self).__init__(*args, **kwargs)
        self.wordcount = {}
        
    def mapper(self,_,line):
        docID, docClass,text = line.split("\t",2)
        text = text.strip()
        for word in word_re.findall(text):
            if word[0]>='a' and word[0]<='z':
                yield word.lower(),1

    def reducer(self,word,counts):
        ov_sum = sum(counts)
        if ov_sum > 2:
            self.wordcount[word] = ov_sum
            yield (word,self.wordcount[word])

if __name__=='__main__':
    Stopwords.run()

Overwriting WordstoKeep.py


In [141]:
!python Stopwords.py "enronemail_1h.txt" > "words_to_keep.txt"

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/Stopwords.z071103.20160905.125739.446816
Running step 1 of 1...
Streaming final output from /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/Stopwords.z071103.20160905.125739.446816/output...
Removing temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/Stopwords.z071103.20160905.125739.446816...


In [145]:
%%writefile NB_threewords.py

from mrjob.job import MRJob
import sys, re

keep_list =[]
with open("words_to_keep.txt") as file:
    for word in file.read().split():
        word_1 = word.strip('"" \t\n\r')
        word_1 = re.sub('[\s+]', '', word_1.lower())
        try:
            int(word_1)
        except:            
            keep_list.append(word_1)
            
def clean_text(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    text= re.sub('[0-9]+',' ',text)
    words = text.split()
    words_n = [x for x in words if x in keep_list]
    for word in words_n :
            cleaned.append(word)
        else:
            pass
    return cleaned

class NB_threewords(MRJob):
          
    def __init__(self, *args, **kwargs):
        super(NB_threewords, self).__init__(*args, **kwargs)
        self.nb_wordstats = {}
        
    def mapper(self,_,line):
        docID, docClass,text = line.split("\t",2)
        words = clean_text(text)
        if docClass == "1":
            yield("PriorsNB","1,0")
            for word in words:
                yield (word, "1,0")        
        else:
            yield("PriorsNB","0,1")
            for word in words:
                yield (word, "0,1")
    
    def reducer(self,word,values):
        spm_total = not_spm_total = 0
        for value in values:
            spm_count, not_spm_count = value.split(",")
            spm_total +=float(spm_count)
            not_spm_total+=float(not_spm_count)
        self.nb_wordstats[word] = [spm_total,not_spm_total]
 

    def reducer_final(self):
        #1.get total wc in spam and not in spam
        spam_words_total = notspam_words_total = spam_docs_total = notspam_docs_total = total_docs = 0
        JM = 0.3
        for word in self.nb_wordstats.keys():
            if word !="PriorsNB":
                spam_words_total+=self.nb_wordstats[word][0]
                notspam_words_total+=self.nb_wordstats[word][1]
            else:
        #2. Calculate and yield prior info from total number of spam and not spam documents and total documents
                spam_docs_total, notspam_docs_total = self.nb_wordstats.get(word)
                total_docs = spam_docs_total + notspam_docs_total
                yield("PriorsNB", ','.join(str(j) for j in [spam_docs_total/total_docs, notspam_docs_total/total_docs])) 
                del self.nb_wordstats[word]
        #4.Get JM proportion of each word 
        for word in self.nb_wordstats.keys():
            yield(word, ','.join(str(j) for j in [(((1-JM)*(self.nb_wordstats[word][0]/spam_words_total)) + (JM *(self.nb_wordstats[word][0] + self.nb_wordstats[word][1])/(spam_words_total + notspam_words_total))),(((1-JM)*(self.nb_wordstats[word][1]/notspam_words_total)) + (JM *(self.nb_wordstats[word][0] + self.nb_wordstats[word][1])/(spam_words_total + notspam_words_total)))])) 
   
    def jobconf(self):
        orig_jobconf = super(NB_threewords, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k2,2rn -k1,1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf

if __name__=='__main__':
    NB_threewords.run()           

Overwriting NB_threewords.py


In [146]:
%reload_ext autoreload
%autoreload 2

from numpy import random
from NBJM_Trainer import NBJM_Trainer 

# STEP 1: Train a mulitnomial Naive Bayes      
trainingData = 'enronemail_1h.txt'

# create an instance of the Trainer class
# and initiatialize it
mr_job = NBJM_Trainer(args=[trainingData])
modelStats={}
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access to the output reducer/reducer_final of 
    # the last step in MRNaiveBayesTrainer
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key, value
        modelStats[key] = value            
    with open('NB_Model.txt', 'w') as f:
        for k in modelStats.keys():
            f.writelines( k + "\t"+ str(modelStats[k]) +"\n")
print modelStats

PriorsNB 0.44,0.56
stock 0.000124893150801,0.000204817111595
limited 0.000519650609807,0.000342966244927
entergyr 9.52864947275e-06,6.191993755e-05
additionally 5.76822506643e-05,7.14485870227e-05
doubts 4.81536011916e-05,9.52864947275e-06
dynamic 1.90572989455e-05,0.0001238398751
externally 9.63072023831e-05,1.90572989455e-05
yellow 4.81536011916e-05,9.52864947275e-06
barraged 4.81536011916e-05,9.52864947275e-06
prefix 4.81536011916e-05,9.52864947275e-06
sleep 9.63072023831e-05,1.90572989455e-05
invovled 1.90572989455e-05,0.0001238398751
railing 4.81536011916e-05,9.52864947275e-06
appetite 9.63072023831e-05,1.90572989455e-05
hate 0.000192614404766,3.8114597891e-05
worked 0.000105835851856,8.09772364955e-05
forget 4.81536011916e-05,9.52864947275e-06
looking 0.000192614404766,3.8114597891e-05
fronts 9.52864947275e-06,6.191993755e-05
founder 4.81536011916e-05,9.52864947275e-06
granting 4.81536011916e-05,9.52864947275e-06
electricity 9.52864947275e-06,6.191993755e-05
turner 0.000105835851

In [144]:
!python NB_Classifier.py --jobconf mapred.reduce.tasks=1 enronemail_1h.txt --file=NB_Model.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.125751.984899
Running step 1 of 1...
Total number of docs is  100
Error rate is  0.17
Streaming final output from /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.125751.984899/output...
Removing temp directory /var/folders/s6/c_ly0ljx5kb4cbtxxx_jr7c54sj_gr/T/NB_Classifier.z071103.20160905.125751.984899...


###3. HW4.6 Benchmark your code with the Python SciKit-Learn Library
Back to Table of Contents
HW4.6.0: Multinomial Naive Bayes using SciKit-Learn
It always a good idea to benchmark your solutions against publicly available libraries such as SciKit-Learn, The Machine Learning toolkit available in Python. In this exercise, we benchmark ourselves against the SciKit-Learn implementation of multinomial Naive Bayes. For more information on this implementation see: http://scikit-learn.org/stable/modules/naive_bayes.html more
In this exercise, please complete the following:
— Run the Multinomial Naive Bayes algorithm (using default settings) from SciKit-Learn over the same training data used in HW1.4.2 and report the misclassification error (please note some data preparation might be needed to get the Multinomial Naive Bayes algorithm from SkiKit-Learn to run over this dataset)
Prepare a table to present your results, where rows correspond to approach used (SkiKit-Learn versus your Hadoop implementation) and the column presents the misclassification error rates (train, validation, testing) — Explain/justify any differences in terms of training error rates over the dataset in HW1.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus the Multinomial Naive Bayes implementation in SciKit-Learn

In [120]:
# For Simplicity save the .txt file as a csv file
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

data = pd.read_csv('Enron_Data.csv')
y = data['Target']
y = np.ravel(y)
X = data.drop('Target',1)
X = X.drop('ID',1)

X = X.T.to_dict().values()
vec = DictVectorizer()
X = vec.fit_transform(X).toarray()

X, X_val, y, y_val = train_test_split(X,y,test_size=0.15,random_state=33)
clf = MultinomialNB()
score = cross_val_score(clf,X,y,cv=5,scoring="accuracy").mean() 
clf.fit(X,y)
print "Error rate during 5 fold cross validated training is" , 1-score
print "Error rate for validation is ", 1 - accuracy_score(y_val, clf.predict(X_val))
print "Confusion matrix for validation is" ,confusion_matrix(y_val, clf.predict(X_val)) 

Error rate during 5 fold cross validated training is 0.399183006536
Error rate for validation is  0.466666666667
Confusion matrix for validation is [[7 0]
 [7 1]]
